In [1]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [20]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [4]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [5]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx',
 'Книга13.xlsx']

In [6]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [10:03<00:00, 54.87s/it]


In [7]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [8]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-11', dayfirst=True, format='mixed'))]

In [77]:
sale = sale.groupby(by = 'Тов', as_index=False)[['Количество', 'Общая сумма']].sum()

In [30]:
stock.head()

,Код товара,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,Размер изделия,Тип изделия 1,...,Бренд,Гендерный признак,Ценовая корзина,Группа цен,Остаток,Чистый вес,Цена изделия на бирке,Скидка на изделии,Запрет скидки,Цена Закупки за 1 шт. (упр сс с НДС за единицу)
0,ТОВ1281578,12-02-0201-11136#ГР,12-02-0201-11136#ГР_ПДК,1058831,ПДК,ПДК СЕРЬГИ,ДК_2544,585,0.0,СЕРЬГИ,...,Значение по умолчанию,женские,СРЕДНЯЯ,ПДК_В,1.0,2.055,94004.0,30.0,Нет,7231.02
1,ТОВ1281716,50-90-0000-27940#375,50-90-0000-27940#375_БК,1353652,БК,БК ПОДВЕС ДЕКОР,БК_1604,375,0.0,ПОДВЕС ДЕКОРАТИВНЫЙ,...,Значение по умолчанию,женские,ДЕШЕВАЯ,ИФ_375,1.0,0.470,8789.0,45.0,Нет,1133.68
2,ТОВ1281720,50-90-0000-24568#375,50-90-0000-24568#375_БК,1465273,БК,БК ПОДВЕС ДЕКОР,БК_1604,375,0.0,ПОДВЕС ДЕКОРАТИВНЫЙ,...,Значение по умолчанию,женские,ДЕШЕВАЯ,ИФ_375,1.0,0.670,12529.0,45.0,Нет,1696.66
3,ТОВ1281721,50-92-0000-27277#375,50-92-0000-27277#375_БК,1954047,БК,БК ПОДВЕС ДЕКОР,БК_1605,375,0.0,ПОДВЕС ДЕКОРАТИВНЫЙ,...,Значение по умолчанию,женские,СРЕДНЯЯ,ИФ_375,1.0,1.009,18887.0,45.0,Нет,2403.03
4,ТОВ1281724,50-90-0000-24029#375,50-90-0000-24029#375_БК,1619144,БК,БК ПОДВЕС ДЕКОР,БК_1604,375,0.0,ПОДВЕС ДЕКОРАТИВНЫЙ,...,Значение по умолчанию,детские,ДЕШЕВАЯ,ИФ_375,1.0,0.321,5984.0,45.0,Нет,770.07


In [10]:
result = stock.drop_duplicates(subset=['Код товара'])

In [24]:
tovs = result['ID Сайта'].dropna().tolist()

In [27]:
images['article'] = images['article'].astype(str)

In [28]:
images = images.query('article in @tovs')

In [31]:
images = images[['article', 'published', 'Ссылка на изображение']]

In [32]:
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']

In [33]:
result = result.merge(images, how='left', on='ID Сайта')

In [37]:
df = stock

In [38]:
stock = df.groupby(by = 'Код товара', as_index=False)['Остаток'].sum()

In [41]:
result = result.merge(stock, how='left', on='Код товара')

In [43]:
result = result.drop(columns=['Остаток_x'])

In [45]:
result.columns = ['Код товара', 'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
    'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Опубликован', 'Ссылка на изображение', 'Остаток']

In [61]:
df['Чистый вес'] = df['Чистый вес'].astype(float)

In [66]:
clear_weight = df.groupby(by = 'Код товара', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.columns = ['Код товара', 'Чистый вес мин', 'Чистый вес медиана', 'Чистый вес макс']
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] - clear_weight['Чистый вес мин']) / ((clear_weight['Чистый вес макс'] + clear_weight['Чистый вес мин']) / 2)

In [69]:
result = result.merge(clear_weight, how='left', on='Код товара')

In [76]:
price_birka = df.groupby(by = 'Код товара', as_index=False)['Цена изделия на бирке'].agg({'min', 'median', 'max'})
price_birka.columns = ['Код товара', 'Цена изделия на бирке мин', 'Цена изделия на бирке медиана', 'Цена изделия на бирке макс']

In [74]:
result = result.merge(price_birka, how='left', on='Код товара')

In [81]:
base_discount = df.groupby(by = 'Код товара', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.columns = ['Код товара', 'Базовая скидка мин', 'Базовая скидка макс']

In [82]:
result = result.merge(base_discount, how='left', on='Код товара')

In [90]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена изделия на бирке'], df['Цена изделия на бирке'] * 0.8 * 0.6 * (1 - df['Скидка на изделии'] / 100)
)

In [94]:
price_skidki = df.groupby(by = 'Код товара', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.columns = ['Код товара', 'Цена со скидками мин', 'Цена со скидками медиана', 'Цена со скидками макс']
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] - price_skidki['Цена со скидками мин']) / ((price_skidki['Цена со скидками макс'] + price_skidki['Цена со скидками мин']) / 2)

In [96]:
result = result.merge(price_skidki, how='left', on='Код товара')

In [102]:
selfprice = df.groupby(by = 'Код товара', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.columns = ['Код товара', 'Себестоимость мин', 'Себестоимость медиана', 'Себестоимость макс']
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] - selfprice['Себестоимость мин']) / ((selfprice['Себестоимость макс'] + selfprice['Себестоимость мин']) / 2)

In [104]:
result = result.merge(selfprice, how='left', on='Код товара')

In [118]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [119]:
markup = df.groupby(by = 'Код товара', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.columns = ['Код товара', 'Наценка мин', 'Наценка медиана', 'Наценка макс']

In [122]:
result = result.merge(markup, how='left', on='Код товара')

In [128]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)

In [129]:
zapret = zapret['Артикул сайта'].tolist()

In [130]:
result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [132]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.17.xlsx')

In [140]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [141]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1']

In [142]:
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [144]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1

In [145]:
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

In [147]:
ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [150]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]

In [152]:
ecom = ecom.query('article in @tovs')

In [155]:
ecom = ecom.groupby(by = 'article', as_index=False)['Цена со скидками Ecom'].agg({'min', 'median', 'max'})

In [165]:
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom мин', 'Цена со скидками Ecom медиан', 'Цена со скидками Ecom макс']
ecom['Цена со скидками Ecom размах, %'] = (ecom['Цена со скидками Ecom макс'] - ecom['Цена со скидками Ecom мин']) / ((ecom['Цена со скидками Ecom макс'] + ecom['Цена со скидками Ecom мин']) / 2)

In [167]:
result = result.merge(ecom, how='left', on='ID Сайта')

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3538889 entries, 0 to 3538888
Data columns (total 25 columns):
 #   Column                                           Dtype  
---  ------                                           -----  
 0   Код товара                                       object 
 1   Артикул поставщика                               object 
 2   Внутренний артикул                               object 
 3   ID Сайта                                         object 
 4   Товарное направление                             object 
 5   Товарная группа                                  object 
 6   Товарный кластер                                 object 
 7   Проба                                            object 
 8   Размер изделия                                   float64
 9   Тип изделия 1                                    object 
 10  Тип изделия 2                                    object 
 11  Тип изделия 3                                    object 
 12  Дизайн        

In [170]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [172]:
summa_ost = df.groupby(by ='Код товара', as_index=False)['Сумма в себестоимости на остатках'].sum()

In [174]:
result = result.merge(summa_ost, how='left', on='Код товара')

In [ ]:
sale = sale.groupby(by ='Тов', as_index=False)['Общая сумма'].agg({'count', 'sum'})

In [181]:
sale.columns = ['Код товара', 'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06']

In [185]:
result = result.merge(sale, how='left', on='Код товара')

In [190]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']

In [194]:
price_per_gramm = df.groupby(by = 'Код товара', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.columns = ['Код товара', 'Цена за грамм мин', 'Цена за грамм медиана', 'Цена за грамм макс']

In [195]:
result = result.merge(price_per_gramm, how='left', on='Код товара')

In [197]:
result.to_csv('result.csv', index=False)

In [219]:
zapret_sk = df.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()

In [220]:
zapret_sk

,Код товара,Запрет скидки
0,ТОВ0115302,11
1,ТОВ0120218,1
2,ТОВ0120219,2
3,ТОВ0120222,3
4,ТОВ0120223,2
...,...,...
140817,ТОВ1335405,58
140818,ТОВ1335481,20
140819,ТОВ1335482,20
140820,ТОВ1336931,20


In [232]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

In [233]:
yes = yes.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()

In [234]:
yes.columns = ['Код товара', 'Количество с запрещенной скидкой']

In [235]:
no.columns = ['Код товара', 'Количество с разрешенной скидкой']

In [236]:
yes = yes.merge(no, how='outer', on='Код товара')

In [239]:
yes

,Код товара,Количество с запрещенной скидкой,Количество с разрешенной скидкой
0,ТОВ0120218,1.0,NaN
1,ТОВ0120219,2.0,NaN
2,ТОВ0120222,3.0,NaN
3,ТОВ0120223,2.0,NaN
4,ТОВ0120224,1.0,NaN
...,...,...,...
140817,ТОВ1335405,NaN,58.0
140818,ТОВ1335481,NaN,20.0
140819,ТОВ1335482,NaN,20.0
140820,ТОВ1336931,NaN,20.0


In [240]:
result = result.merge(yes, how='left', on='Код товара')

In [244]:
result = result.drop_duplicates(subset=['Код товара'])

In [245]:
result.to_excel('Исследование цен доработанный.xlsx', index=False)